In [1]:
from __future__ import print_function, division
import numpy as np
import json
import time
import scipy
import scipy.sparse
from trimesh import TriMesh
import glob
from numpy import *
np.set_printoptions(linewidth=2000, suppress=True)
from Extract_Transformation_matrix_minimize_SVD_singular_value_allPoses import *

In [2]:
#### read obj file
# base_dir="./datas/cube4/poses-1/"
base_dir="../models/cube4/poses-1/"
filenames=glob.glob(base_dir+"*.obj")
print(filenames)
meshes_List=[]
for i in range(len(filenames)):
    meshes_List.append(TriMesh.FromOBJ_FileName(filenames[i]))

mesh0=meshes_List[0]
mesh1_list=meshes_List[1:]

groundtruth_names=glob.glob(base_dir+"cube-*.DMAT")
print(groundtruth_names)

gt_all=None
for i in range(len(groundtruth_names)):
    gt=load_DMAT(groundtruth_names[i])
    gt=gt.T
    gt=gt.reshape((-1,4,3))
    gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
    if gt_all is None:
        gt_all=gt
    else:
        gt_all=np.hstack((gt_all, gt))
    

print(gt_all.shape)

ind=np.arange(10)


vertices1=vertices1_temp=np.hstack((np.asarray(mesh0.vs),np.ones((len(mesh0.vs),1))))
vertices2=vertices2_temp=np.asarray(mesh1_list[0].vs)
for i in range(1,len(mesh1_list)):
    vertices1=np.hstack((vertices1, vertices1_temp))
    vertices2=np.hstack((vertices2, np.asarray(mesh1_list[i].vs)))
print(vertices1.shape) 
print(vertices2.shape) 

P = len(mesh1_list)
M=12*P

['../models/cube4/poses-1/cube-1.obj', '../models/cube4/poses-1/cube-2.obj', '../models/cube4/poses-1/cube-3.obj', '../models/cube4/poses-1/cube-4.obj']
['../models/cube4/poses-1/cube-1.DMAT', '../models/cube4/poses-1/cube-2.DMAT', '../models/cube4/poses-1/cube-3.DMAT', '../models/cube4/poses-1/cube-4.DMAT']
(1538, 48)
(1538, 12)
(1538, 9)


In [3]:
#### solve for transformation per vertex, with smoothness regularization and transformation matrix L2 norm regularization.
output_prefix=base_dir+"Vertex_tramsformation_matrix_recovering"
weights = {'W_svd': 2.0, 
           'W_rotation': 0.01, 
           'W_rotation1': 0.0, 
           'W_rotation2': 0.0, 
           'W_translation':0.0, 
           'W_spatial': 0.0
          }


start=time.time()

print("solving transformation matrix")



x0=np.ones(len(vertices1)*M)/M
    
# x0=np.zeros(len(vertices1)*M)

# x0=gt_all.ravel()

# #### use identity matrix as input
# xi=np.hstack((np.identity(3),np.zeros((3,1)))).reshape((1,-1))
# print xi
# x0=np.repeat(xi, len(vertices0), 0).reshape(-1)



res=run_one(mesh0, mesh1_list, output_prefix, weights, x0)
res=res.reshape((len(mesh0.vs),-1))
end=time.time()
print("using time: ", end-start)
print(res.shape)

solving transformation matrix
(1538, 12)
(1538, 9)
############################
loop:  0
{'W_rotation': 0.01, 'W_svd': 2.0, 'W_translation': 0.0, 'W_spatial': 0.0, 'W_rotation1': 0.0, 'W_rotation2': 0.0}
round:  1
True
singular values:  [ 33.62749286  17.95759961  12.48248001  11.13223229   9.15966144   8.08861349   3.77464853   3.2059284    2.87542569   2.20002607   1.95953457   1.77890307   1.55513286   1.4340664    0.94401797   0.82225942   0.78791391   0.64575605   0.60252002   0.54262277   0.48231961   0.44309014   0.39223934   0.31096606   0.28007423   0.27093221   0.2203707    0.21225045   0.18738771   0.16183247   0.14291535   0.14015507   0.10335878   0.08289098   0.06552393] 2.37940980356e-05 0
recontruction error:  0.00109160828329
round:  2
True
singular values:  [ 33.6225838   17.95581385  12.48247478  11.13270213   9.15927699   8.08901228   3.77409576   3.20612518   2.87514793   2.20021927   1.96362213   1.77856191   1.55466025   1.43432785   0.94411855   0.82201586   0.7

LinAlgError: SVD did not converge

In [ ]:
print res.shape
L=res-res.mean(axis=0).reshape((1,-1))

eigenVal,eigenVec=np.linalg.eig(L.T.dot(L))

print(sorted(eigenVal))

In [ ]:
gtL=gt_all-gt_all.mean(axis=0).reshape((1,-1))

gteigenVal,gteigenVec=np.linalg.eig(gtL.T.dot(gtL))

print(sorted(gteigenVal))

In [ ]:
print(res.shape)

L=res-res.mean(axis=0).reshape((1,-1))

s = np.linalg.svd(L, full_matrices=True, compute_uv=False)
print(s)

In [ ]:
gtL=gt_all-gt_all.mean(axis=0).reshape((1,-1))

s = np.linalg.svd(gtL, full_matrices=True, compute_uv=False)
print(s)

In [ ]:
temp1=res.reshape((-1,3,4))[:,:,:3]
temp2=temp1.transpose((0,2,1))
identities=np.repeat(np.identity(3).reshape((1,-1)), len(temp1), 0).ravel()
print(abs((temp1[:,:,:,np.newaxis]*temp2[:,np.newaxis,:,:]).sum(axis=-2).ravel()-identities).max())

In [ ]:
temp1=gt_all.reshape((-1,3,4))[:,:,:3]
temp2=temp1.transpose((0,2,1))
identities=np.repeat(np.identity(3).reshape((1,-1)), len(temp1), 0).ravel()
print(abs((temp1[:,:,:,np.newaxis]*temp2[:,np.newaxis,:,:]).sum(axis=-2).ravel()-identities).max())

In [ ]:
#### check non_diagnal_sum of RTR is 0.
temp1=res.reshape((-1,3,4))[:,:,:3]
temp2=temp1.transpose((0,2,1))
RTR=(temp1[:,:,:,np.newaxis]*temp2[:,np.newaxis,:,:]).sum(axis=-2).ravel() #### per vertex R^T*R.

RTRsqure=np.square(RTR)
inds=np.repeat(np.array([[0,4,8]]), len(temp1), 0)+ np.arange(len(temp1)).reshape((-1,1))*9  #### diganl element index in ravel array.

diagnal_term=np.square(RTR[inds.ravel()]-np.ones(len(inds.ravel()))).sum()
non_diagnal_sum_square=RTRsqure.sum()-RTRsqure[inds.ravel()].sum()

print(diagnal_term)
print(non_diagnal_sum_square)

all_inds=np.arange(len(temp1)*9)
non_diagnal_inds=np.asarray(list(set(list(all_inds))-set(list(inds.ravel()))))

print(abs(RTR[non_diagnal_inds]).max())
print(np.median(abs(RTR[non_diagnal_inds])))
print(RTR[inds.ravel()].max())
print(RTR[inds.ravel()].min())

In [ ]:
#### check non_diagnal_sum of RTR is 0.
temp1=gt_all.reshape((-1,3,4))[:,:,:3]
temp2=temp1.transpose((0,2,1))
RTR=(temp1[:,:,:,np.newaxis]*temp2[:,np.newaxis,:,:]).sum(axis=-2).ravel() #### per vertex R^T*R.

RTRsqure=np.square(RTR)
inds=np.repeat(np.array([[0,4,8]]), len(temp1), 0)+ np.arange(len(temp1)).reshape((-1,1))*9  #### diganl element index in ravel array.

diagnal_term=np.square(RTR[inds.ravel()]-np.ones(len(inds.ravel()))).sum()
non_diagnal_sum_square=RTRsqure.sum()-RTRsqure[inds.ravel()].sum()

print(diagnal_term)
print(non_diagnal_sum_square)

all_inds=np.arange(len(temp1)*9)
non_diagnal_inds=np.asarray(list(set(list(all_inds))-set(list(inds.ravel()))))

print(abs(RTR[non_diagnal_inds]).max())
print(np.median(abs(RTR[non_diagnal_inds])))
print(RTR[inds.ravel()].max())
print(RTR[inds.ravel()].min())

In [ ]:
print(abs(res-gt_all).max())
print(res.ravel()[abs(res-gt_all).argmax()])
print(gt_all.ravel()[abs(res-gt_all).argmax()])
print(np.sqrt(np.square(res-gt_all).mean()))